# 8.

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split


mnist = datasets.fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000,
                                                    stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=50000,
                                                  stratify=y_train,
                                                  random_state=42)
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(50000, 784)
(10000, 784)
(10000, 784)


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

forest_grid = {
    'n_estimators': [130],
    'criterion': ['gini'],
}
forest_search = GridSearchCV(RandomForestClassifier(random_state=42), forest_grid,
                             scoring='accuracy', verbose=1)
forest_search.fit(X_train, y_train)

y_pred = forest_search.best_estimator_.predict(X_val)
# {'criterion': 'gini', 'n_estimators': 130}
print(forest_search.best_params_)
print(accuracy_score(y_val, y_pred))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
{'criterion': 'gini', 'n_estimators': 130}
0.9718


In [4]:
from sklearn.ensemble import ExtraTreesClassifier


extra_trees_grid = {
    'n_estimators': [85, 100, 115, 130],
    'criterion': ['gini', 'entropy', 'log_loss'],
}
extra_trees_search = GridSearchCV(ExtraTreesClassifier(), extra_trees_grid,
                                  scoring='accuracy', verbose=1)
extra_trees_search.fit(X_train, y_train)

y_pred = extra_trees_search.best_estimator_.predict(X_val)
print(extra_trees_search.best_params_)
print(accuracy_score(y_val, y_pred))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


svm_pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('prediction', SVC(random_state=42))
])
svm_grid= {
    'prediction__C': [1, 5, 10, 20, 50, 100],
    'prediction__kernel': ['linear', 'poly', 'rbf'],
    'prediction__degree': [2, 3, 4, 5],
}
svm_search = GridSearchCV(svm_pipe, svm_grid, scoring='accuracy', verbose=1)
svm_search.fit(X_train, y_train)

y_pred = extra_trees_search.best_estimator_.predict(X_val)
print(svm_search.best_params_)
print(accuracy_score(y_val, y_pred))

In [ ]:
from sklearn.ensemble import VotingClassifier


voting_clf = VotingClassifier(
    estimators=[('svc', svm_search.best_estimator_),
                ('et', extra_trees_search.best_estimator_),
                ('rt', forest_search.best_estimator_)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_val)
print(accuracy_score(y_val, y_pred))

In [ ]:
estimators = (svm_search.best_estimator_, extra_trees_search.best_estimator_,
              forest_search.best_estimator_, voting_clf)
for model in estimators:
    y_pred = model.predict(X_test)
    print(model.__class__.__name__, accuracy_score(y_test, y_pred))

# 9.

In [ ]:
import numpy as np


val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)
for index, model in enumerate(estimators):
    val_predictions[:, index] = model.predict(X_test)

print(val_predictions)

In [ ]:
forest_blender = RandomForestClassifier(oob_score=True, random_state=42)
forest_blender.fit(val_predictions, y_val)

y_pred = forest_blender.predict(X_test)
print(accuracy_score(y_test, y_pred))